In [47]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from sqlalchemy import create_engine

# How much time is spent on Education vs. PISA Score by Country

# Extract information

In [2]:
# read the time spent on different activities
excel_file = "Resources/Time-Use-in-OECD-Countries-OECD.xlsx"
time_use_df = pd.read_excel(excel_file)
time_use_df.head()

,Country,Category,Time (hours)
0,Australia,Paid work,211.146630
1,Austria,Paid work,279.532268
2,Belgium,Paid work,194.476452
3,Canada,Paid work,268.660610
4,Denmark,Paid work,199.771596


In [44]:
time_use_df["Category"].unique()

array(['Paid work', 'Education', 'Care for household members ',
       'Housework', 'Shopping', 'Other unpaid work & volunteering',
       'Sleep', 'Eating and drinking', 'Personal care', 'Sports',
       'Attending events', 'Seeing friends', 'TV and Radio',
       'Other leisure activities'], dtype=object)

In [5]:
# read the PISA score from each country (educational score)
csv_file = "Resources/pisa-test-score-mean-performance-on-the-science-scale.csv"
pisa_df = pd.read_csv(csv_file)
pisa_df.head()

,Entity,Code,Year,PISA: Mean performance on the science scale
0,Albania,ALB,2000,376.453214
1,Albania,ALB,2009,390.695746
2,Albania,ALB,2012,397.373214
3,Albania,ALB,2015,427.225000
4,Algeria,DZA,2015,375.745100


# Transform

In [6]:
# Clean dataframe to have only the most recent score data
year = pisa_df["Year"] == 2015
pisa_2015_df = pisa_df.loc[year]
pisa_2015_df.head()

,Entity,Code,Year,PISA: Mean performance on the science scale
3,Albania,ALB,2015,427.2250
4,Algeria,DZA,2015,375.7451
9,Argentina,ARG,2015,432.2262
15,Australia,AUS,2015,509.9939
21,Austria,AUT,2015,495.0375


In [7]:
# Get only the time spent on education by country
education = time_use_df["Category"] == "Education"
time_use_education_df = time_use_df.loc[education]
time_use_education_df.head()

,Country,Category,Time (hours)
33,Australia,Education,27.018763
34,Austria,Education,26.897949
35,Belgium,Education,41.277172
36,Canada,Education,35.980637
37,Denmark,Education,25.044998


In [14]:
# Merge two data sets into one by country
education_df = time_use_education_df.merge(pisa_2015_df, left_on="Country", right_on="Entity")
education_df.head()

,Country,Category,Time (hours),Entity,Code,Year,PISA: Mean performance on the science scale
0,Australia,Education,27.018763,Australia,AUS,2015,509.9939
1,Austria,Education,26.897949,Austria,AUT,2015,495.0375
2,Belgium,Education,41.277172,Belgium,BEL,2015,501.9997
3,Canada,Education,35.980637,Canada,CAN,2015,527.7047
4,Denmark,Education,25.044998,Denmark,DNK,2015,501.9369


In [15]:
# delete columns that we don't need
education_df.drop(['Entity', 'Code', 'Category'], axis=1, inplace=True)
education_df.head()

,Country,Time (hours),Year,PISA: Mean performance on the science scale
0,Australia,27.018763,2015,509.9939
1,Austria,26.897949,2015,495.0375
2,Belgium,41.277172,2015,501.9997
3,Canada,35.980637,2015,527.7047
4,Denmark,25.044998,2015,501.9369


In [22]:
education_df.columns

Index(['Country', 'Time (hours)', 'Year',
       'PISA: Mean performance on the science scale'],
      dtype='object')

# Load

In [48]:
# PostgreSQL Connection
project_db = "etl_project_db" #poner nombre de base de datos que creemos
rds_connection_string = f"postgres:1234@localhost:5432/{project_db}"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

[]

In [19]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [27]:
# Define database
db = client.etl_project_db

# create list to insert
education_list = []

In [28]:
# iterate the dataframe to populate list with only the relevant data needed
for index,row in education_df.iterrows():
    education = {
        "Country": row["Country"],
        "Time (hours)": row['Time (hours)'],
        "PISA: Mean performance on the science scale": row["PISA: Mean performance on the science scale"]
    }
    education_list.append(education)

In [29]:
# create collection and insert many results
db.score_time_education.insert_many(education_list)

# Alcohol Consumption vs. Eating and Drinking time spent by Country 

# Extract

In [32]:
# read the Alcohol consumption by Country (pure alcohol in liters)
xmart_csv = "Resources/xmart.csv"
xmart_df = pd.read_csv(xmart_csv)
xmart_df.head()

,Country,Data Source,Beverage Types,2018,2017,2016,2015,2014,2013,2012,2011,2010
0,Afghanistan,Data source,All types,0.01,0.01,0.02,0.0,0.01,0.01,0.01,0.01,0.01
1,Afghanistan,Data source,Beer,0.00,0.00,0.00,0.0,0.00,0.01,0.01,0.01,0.01
2,Afghanistan,Data source,Wine,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
3,Afghanistan,Data source,Spirits,0.01,0.01,0.01,0.0,0.01,0.00,0.00,0.00,0.00
4,Afghanistan,Data source,Other alcoholic beverages,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00


# Transform

In [34]:
eat_var = time_use_df["Category"] == 'Eating and drinking'
eat_drink_time_df = time_use_df.loc[eat_var]
eat_drink_time_df.head()

,Country,Category,Time (hours)
231,Australia,Eating and drinking,89.061849
232,Austria,Eating and drinking,78.836406
233,Belgium,Eating and drinking,98.516408
234,Canada,Eating and drinking,64.819713
235,Denmark,Eating and drinking,119.153410


In [36]:
all_var = xmart_df["Beverage Types"] == ' All types'
consumption_all_df = xmart_df.loc[all_var]
consumption_all_df.head()

,Country,Data Source,Beverage Types,2018,2017,2016,2015,2014,2013,2012,2011,2010
0,Afghanistan,Data source,All types,0.01,0.01,0.02,0.00,0.01,0.01,0.01,0.01,0.01
5,Albania,Data source,All types,4.70,4.75,4.67,4.54,4.33,4.54,4.74,5.42,5.28
10,Algeria,Data source,All types,0.58,0.58,0.60,0.58,0.54,0.50,0.43,0.40,0.35
15,Andorra,Data source,All types,9.75,9.93,9.94,9.85,9.89,10.07,10.34,10.46,10.58
20,Angola,Data source,All types,5.35,5.47,5.38,5.94,7.49,7.01,7.14,6.95,6.83


In [37]:
consumption_all_df["Avg"] = consumption_all_df[["2010","2011","2012","2013","2014","2015"]].mean(axis=1)
consumption_all_df.head()

<ipython-input-37-79f839545059>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consumption_all_df["Avg"] = consumption_all_df[["2010","2011","2012","2013","2014","2015"]].mean(axis=1)


,Country,Data Source,Beverage Types,2018,2017,2016,2015,2014,2013,2012,2011,2010,Avg
0,Afghanistan,Data source,All types,0.01,0.01,0.02,0.00,0.01,0.01,0.01,0.01,0.01,0.008333
5,Albania,Data source,All types,4.70,4.75,4.67,4.54,4.33,4.54,4.74,5.42,5.28,4.808333
10,Algeria,Data source,All types,0.58,0.58,0.60,0.58,0.54,0.50,0.43,0.40,0.35,0.466667
15,Andorra,Data source,All types,9.75,9.93,9.94,9.85,9.89,10.07,10.34,10.46,10.58,10.198333
20,Angola,Data source,All types,5.35,5.47,5.38,5.94,7.49,7.01,7.14,6.95,6.83,6.893333


In [39]:
# Merge two data sets into one by country
consumption_df = eat_drink_time_df.merge(consumption_all_df, on="Country")
consumption_df.head()

,Country,Category,Time (hours),Data Source,Beverage Types,2018,2017,2016,2015,2014,2013,2012,2011,2010,Avg
0,Australia,Eating and drinking,89.061849,Data source,All types,9.51,9.49,9.79,9.68,9.90,10.00,10.03,10.30,10.52,10.071667
1,Austria,Eating and drinking,78.836406,Data source,All types,11.80,11.70,11.70,11.60,12.20,11.60,12.10,11.90,12.10,11.916667
2,Belgium,Eating and drinking,98.516408,Data source,All types,9.42,9.42,9.42,10.36,10.57,10.33,10.09,10.14,10.27,10.293333
3,Canada,Eating and drinking,64.819713,Data source,All types,8.20,8.20,8.10,8.00,8.00,8.20,8.30,8.20,8.30,8.166667
4,Denmark,Eating and drinking,119.153410,Data source,All types,9.31,9.30,9.51,9.42,9.56,9.44,9.23,10.45,10.27,9.728333


In [41]:
consumption_df.columns

Index(['Country', 'Category', 'Time (hours)', 'Data Source', 'Beverage Types',
       '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010',
       'Avg'],
      dtype='object')

In [42]:
consumption_list = []

for idx,row in consumption_df.iterrows():
    cons_dict = {
        "Country": row["Country"],
        "time_spent": row["Time (hours)"],
        "avg_consumption": row["Avg"]
    }
    consumption_list.append(cons_dict)


[{'Country': 'Australia',
  'time_spent': 89.06184850590688,
  'avg_consumption': 10.071666666666667},
 {'Country': 'Austria',
  'time_spent': 78.83640556233911,
  'avg_consumption': 11.916666666666666},
 {'Country': 'Belgium',
  'time_spent': 98.5164083643487,
  'avg_consumption': 10.293333333333333},
 {'Country': 'Canada',
  'time_spent': 64.81971259158588,
  'avg_consumption': 8.166666666666666},
 {'Country': 'Denmark',
  'time_spent': 119.15340996707874,
  'avg_consumption': 9.728333333333333},
 {'Country': 'Estonia',
  'time_spent': 78.99620695899661,
  'avg_consumption': 16.62333333333333},
 {'Country': 'Finland',
  'time_spent': 80.64583,
  'avg_consumption': 9.193333333333333},
 {'Country': 'France',
  'time_spent': 132.75710778104158,
  'avg_consumption': 12.089999999999998},
 {'Country': 'Germany',
  'time_spent': 94.96315,
  'avg_consumption': 11.170000000000002},
 {'Country': 'Greece',
  'time_spent': 127.53410855795852,
  'avg_consumption': 7.335},
 {'Country': 'Hungary',


In [45]:
# create collection and insert many results
db.time_alcohol_consumption.insert_many(education_list)